In [265]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np

In [266]:
# parâmetros de entrada
# X: dataset com os dados a serem agrupados
# eps: raio de vizinhança
# minPts: número mínimo de pontos
# out:

X = pd.read_csv('data\iris1-10.csv', delimiter=',')
X.drop('class', axis=1, inplace=True)
X.head()

eps = 0.4
minPts = 2

In [267]:
def showCluster(n, p):
    print("".join("=" for i in range(20)))
    print("Cluster ", n, ": " , "{", p, "}")
    print("".join("=" for i in range(20)))

In [268]:
def showRuidos(r):
    print("".join("=" for i in range(20)))
    print("Ruídos:" , "{", r, "}")
    print("".join("=" for i in range(20)))

In [269]:
# Calcular a distância euclidiana entre os pontos: https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html
Y_dist = pdist(X, 'euclidean')
Y_dist = squareform(Y_dist) # converte para matriz
# X.shape # (150, 4) -> 150 pontos com 4 dimensões
# Y.shape # (150, 150) -> 11175 distâncias
print(len(Y_dist))

10


In [270]:
# Vetor para armazenar pontos já visitados
visitados = []

# Vetor para armazenar pontos candidatos a estarem no cluster
candidatos = []

# Vetor com os ruídos
ruidos = []

# Vetor com os pontos do cluster
cluster = []

# Pontos a serem visitados
restantes = []
restantes += range(0, 10)

# Número de clusters
nClusters = 0

In [271]:
while len(restantes) > 0:
    # Escolher ponto aleatório na faixa entre 0 e 150 (exclusive)
    p = np.random.randint(0, 10)

    for i in range(0, 10):
        # Verificar se o ponto já foi visitado
        if p not in visitados:
            # Verificar se o ponto está dentro do raio de vizinhança
            if Y_dist[p, i] <= eps:
                candidatos.append(i)
            else:
                ruidos.append(i)
        
    visitados.append(p)
    
    restantes = [pts for pts in restantes if pts not in ruidos]
           
    if len(candidatos) >= minPts:    
        # Inicializar identificador do novo cluster
        nClusters += 1
                
        # Adicionar os pontos candidatos ao cluster
        cluster = candidatos
        
        # Adiciona p ao cluster
        cluster.append(p)
        showCluster(nClusters, p)
        
        # Adicionar à lista de visitados os pontos do cluster
        visitados.append(cluster)
        
        # Remover os pontos do cluster da lista de restantes        
        restantes = [pts for pts in restantes if pts not in cluster]

showRuidos(ruidos)

Cluster  1 :  { 6 }
Ruídos: { [0, 1, 2, 3, 4, 6, 7, 8, 9, 0, 1, 4, 5, 7, 8, 9] }
